In [1]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import csv
from torchvision import transforms
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 이미지 크기를 64x64로 조정
    transforms.ToTensor(),         # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

In [4]:
class ImageComparisonModel(nn.Module):
    def __init__(self):
        super(ImageComparisonModel, self).__init__()
        
        # Stream 1 for the first image modality
        self.stream1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Stream 2 for the second image modality
        self.stream2 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fusion and decision mechanism
        self.comparison = nn.Sequential(
            nn.Linear(16*16*256*2, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),  # Binary classification
        )
        
    def forward(self, x1, x2):
        x1 = self.stream1(x1)
        x2 = self.stream2(x2)
        
        # Flatten the features from both streams
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        
        # Concatenate the features
        x = torch.cat((x1, x2), dim=1)
        
        # Pass through the comparison mechanism
        x = self.comparison(x)
        # Apply softmax to get probabilities
        # x = F.softmax(x, dim=1)
        return x

In [5]:
# 저장된 .pth 파일의 경로
saved_model_path = "Check_Model.pth"

In [6]:
# 저장된 모델 클래스의 인스턴스 생성
loaded_model = ImageComparisonModel()

In [7]:
# 저장된 모델의 가중치 불러오기
loaded_model.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [8]:
# 모델을 평가 모드로 설정
loaded_model.eval()

ImageComparisonModel(
  (stream1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (stream2): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNo

In [37]:
# 경로 설정
root_dir1 = 'data_test/chemdraw_LastTestSet'
root_dir2 = 'data_test/chemdraw_LastTestSet_smlies'
root_dir3 = 'data_test/chemdraw_LastTestSet_smlies_wrong'

In [38]:
# 폴더 내 모든 이미지 파일 경로 가져오기
image_files1 = [os.path.join(root_dir1, filename) for filename in os.listdir(root_dir1) if filename.endswith(('.png', '.jpg', '.jpeg'))]
image_files2 = [os.path.join(root_dir2, filename) for filename in os.listdir(root_dir2) if filename.endswith(('.png', '.jpg', '.jpeg'))]
image_files3 = [os.path.join(root_dir3, filename) for filename in os.listdir(root_dir3) if filename.endswith(('.png', '.jpg', '.jpeg'))]

In [39]:
# 이미지 파일을 모두 불러와서 리스트에 저장
images1 = [Image.open(img_path).convert("RGB") for img_path in image_files1]
images2 = [Image.open(img_path).convert("RGB") for img_path in image_files2]
images3 = [Image.open(img_path).convert("RGB") for img_path in image_files3]

In [40]:
# 모델을 CUDA 장치로 이동
loaded_model.to(device)

ImageComparisonModel(
  (stream1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (stream2): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNo

In [41]:
# 결과 저장을 위한 리스트
results = []

# 예측 레이블과 정답 레이블을 저장할 리스트
predicted_labels = []
true_labels = []

# 레이블 지정
label = 1

# tqdm을 사용하여 진행 상황 표시
for img1_name, img1 in zip(image_files1, tqdm(images1, desc="Processing images", unit="image")):
    # img1과 대응되는 이미지2를 찾음
    img1_idx = image_files1.index(img1_name)  # img1의 인덱스
    img2_name, img2 = image_files2[img1_idx], images2[img1_idx]  # 대응되는 img2의 이름과 이미지

    # 이미지를 모델 입력에 맞게 전처리
    img1_tensor = transform(img1).unsqueeze(0).to(device)  # GPU로 이동
    img2_tensor = transform(img2).unsqueeze(0).to(device)  # GPU로 이동

    # 이미지를 비교하여 예측 수행
    with torch.no_grad():
        output = loaded_model(img1_tensor, img2_tensor)
        similarity = torch.sigmoid(output).item()

    # 결과 저장
    results.append([img1_name, img2_name, similarity])
    # 예측 레이블과 정답 레이블 저장
    predicted_labels.append(1 if similarity > 0.5 else 0)
    true_labels.append(label)  # 레이블은 1로 설정

print("완료되었습니다.")


Processing images: 100%|█████████▉| 499/500 [00:02<00:00, 213.81image/s]

완료되었습니다.


In [42]:
# 결과를 DataFrame으로 변환
df = pd.DataFrame(results, columns=['image1_name', 'image2_name', 'similarity'])

# 예측 레이블 및 정답 레이블 추가
df['predicted_label'] = predicted_labels
df['true_label'] = true_labels

# 결과를 CSV 파일로 저장
output_csv_path = "image_similarity_results_wrong.csv"
df.to_csv(output_csv_path, index=False)

print(f"Results saved to {output_csv_path}")

Results saved to image_similarity_results_wrong.csv


In [43]:
# 혼동 행렬 계산
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[  0   0]
 [109 391]]


In [44]:
# 정밀도 계산
precision = precision_score(true_labels, predicted_labels)

# 재현율 계산
recall = recall_score(true_labels, predicted_labels)

# F1 스코어 계산
f1 = f1_score(true_labels, predicted_labels)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 1.0
Recall: 0.782
F1 Score: 0.877665544332211
